In [ ]:
# coding=utf-8
from __future__ import print_function, absolute_import, unicode_literals
from gm.api import *
import sys
import MySQLdb
import codecs
from datetime import *
import time

sys.path.append('../')
from DB import *


set_token('a0998908534d317105b2184afbe436a4104dc51b')
        
class StockData():
    def __init__(self):
        self.db = DB()
        
    def getStockList(self):
        return self.db.getStockList()
    
    def getSymbol(self, code):
        if code[0:1] == "6":
            return "SHSE"
        return "SZSE"
    
    def queryAllStockBar(self, md, code):
        startdate = self.db.getStockLatestDate(code)
        if startdate is None:
            startdate = self.db.getStockStartDate(code)
        else:
            startdate = startdate + timedelta(days=1)
        enddate = datetime.now()+timedelta(days=1)
        print(startdate, enddate)
        bars = md.get_dailybars(self.getSymbol(code) + "." + code, startdate.strftime("%Y-%m-%d"), enddate.strftime("%Y-%m-%d"))
        return bars
    
    def saveBars(self, bars):
        for bar in bars:
            self.db.addStockDailyBar(bar)
        print("insert %d bars" % len(bars))

class FundamentalsData():
    def __init__(self):
        self.db = DB()

    def getSymbol(self, code):
        if code[0:1] == "6":
            return "SHSE"
        return "SZSE"

    def getFundamentals(self, code, table, fieldstr):
        startdate = self.db.getLastFundamentalsDate(code, table)
        enddate = datetime.now()+timedelta(days=1)
        print(startdate, enddate, self.getSymbol(code)+ "." +code)
        results = get_fundamentals(table=table, symbols=self.getSymbol(code) + "." + code, 
            start_date=startdate, end_date=enddate, limit=10000,
                fields=fieldstr)
        return results
    def removeDuplicate(self, fundamentals):
        results = {}
        for fundamental in fundamentals:
            key = fundamental['symbol'] + fundamental['pub_date'].strftime("%Y-%m-%d") + fundamental['end_date'].strftime("%Y-%m-%d")
            if key in results and not self.isSame(fundamental, results[key]):
                print("*****", fundamental, results[key])
            else:
                results[key] = fundamental
        return results.values()
    def isSame(self, item1, item2):
        keys = item1.keys()

        for key in keys:
            if key not in item2 or item1[key] != item2[key]:
                return False
        return True
    def saveFundamentals(self, code, fundamentals, table, fieldstr):
#         for fundamental in fundamentals:
        fundamentals = self.removeDuplicate(fundamentals)
        if len(fundamentals) > 0:
            self.db.addFundamental(code, fundamentals, table, fieldstr)
            print("insert %d fundamentals" % len(fundamentals))
        else:
            print("%s is up to date" % code)


fieldstr = 'ACCHELDFORS,ACCOPAYA,ACCORECE,ACCREXPE,ACCUDEPR,ACTITRADSECU,ACTIUNDESECU,ADVAPAYM,AVAISELLASSE,BDSPAYA,BDSPAYAPERBOND,BDSPAYAPREST,CAPISURP,CENBANKBORR,COMASSE,CONSPROG,COPEPOUN,COPEWITHREINRECE,COPEWORKERSAL,CURFDS,CURTRANDIFF,DEFEINCOTAXLIAB,DEFEREVE,DEFETAXASSET,DEPOSIT,DERIFINAASSET,DERILIAB,DEVEEXPE,DIVIDRECE,DIVIPAYA,DOMETICKSETT,DUENONCLIAB,ENGIMATE,EQUIINVE,EXPECURRLIAB,EXPENONCLIAB,EXPINONCURRASSET,EXPOTAXREBARECE,FDSBORR,FIXEDASSECLEA,FIXEDASSEIMMO,FIXEDASSEIMPA,FIXEDASSENET,FIXEDASSENETW,GENERISKRESE,GOODWILL,HOLDINVEDUE,HYDRASSET,INSUCONTRESE,INTAASSET,INTELPAY,INTELRECE,INTEPAYA,INTERECE,INTETICKSETT,INVE,INVEPROP,LCOPEWORKERSAL,LENDANDLOAN,LIABHELDFORS,LOGPREPEXPE,LONGBORR,LONGDEFEINCO,LONGPAYA,LONGRECE,MARGRECE,MARGREQU,MINYSHARRIGH,NOTESPAYA,NOTESRECE,OCL,OTHEQUIN,OTHERCURRASSE,OTHERCURRELIABI,OTHERFEEPAYA,OTHERLONGINVE,OTHERNONCASSE,OTHERNONCLIABI,OTHERPAY,OTHERRECE,PAIDINCAPI,PARESHARRIGH,PERBOND,PLAC,PREMRECE,PREP,PREPEXPE,PREST,PRODASSE,PUBLISHDATE,PURCRESAASSET,REINCONTRESE,REINRECE,RESE,RIGHAGGR,SELLREPASSE,SETTRESEDEPO,SFORMATCURRASSE,SFORMATCURRELIABI,SFORMATNONCASSE,SFORMATNONCLIAB,SFORMATPARESHARRIGH,SFORMATRIGHAGGR,SFORMATTOTASSET,SFORMATTOTLIAB,SFORMATTOTLIABSHAREQUI,SHORTTERMBDSPAYA,SHORTTERMBORR,SMERGERCURRASSE,SMERGERCURRELIABI,SMERGERNONCASSE,SMERGERNONCLIAB,SMERGERPARESHARRIGH,SMERGERRIGHAGGR,SMERGERTOTASSET,SMERGERTOTLIAB,SMERGERTOTLIABSHAREQUI,SPECPAYA,SPECRESE,SUBSRECE,SUNEVENASSETLIABEUQI,SUNEVENCURRASSE,SUNEVENCURRELIABI,SUNEVENNONCASSE,SUNEVENNONCLIAB,SUNEVENPARESHARRIGH,SUNEVENRIGHAGGR,SUNEVENTOTASSET,SUNEVENTOTLIAB,SUNEVENTOTLIABSHAREQUI,TAXESPAYA,TOPAYCASHDIVI,TOTALCURRLIAB,TOTALNONCASSETS,TOTALNONCLIAB,TOTASSET,TOTCURRASSET,TOTLIAB,TOTLIABSHAREQUI,TRADFINASSET,TRADFINLIAB,TRADSHARTRAD,TREASTK,UNDIPROF,UNREINVELOSS,UNSEG,WARLIABRESE'
table = 'balance_sheet'
providor = FundamentalsData()
stockData = StockData()
for code in stockData.getStockList():
    print("##", code, "##")

    fundamentals = providor.getFundamentals(code, table, fieldstr)
    providor.saveFundamentals(code, fundamentals, table, fieldstr)
